#PRACTICA MLOps

###Instalación de Librerias

In [ ]:
%%capture
#instalamos libreria tensoflow
!pip install notebook tensorflow


In [ ]:
#instalamos libreria wandb
%%capture
!pip install wandb

In [ ]:
%%capture
#instalamos libreria mlflow
!pip install mlflow pyngrok

In [ ]:
#Nos logueamos con nuestra cuenta para poder registrar metricas
! wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### Importación librerias

In [ ]:
#importacion de liberias necesarias para entrenamiento e implementacion de nuestro modelo
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np
import mlflow
import mlflow.tensorflow
import wandb
from wandb.integration.keras import WandbMetricsLogger
import kagglehub
import cv2
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten, Dense, Reshape, Conv2D, MaxPooling2D
import os
import tempfile


### CARGA CONJUNTO DATOS

In [ ]:
#montamos volumen de google drive donde almacenaremos el conjunto de datos seleccionado
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%%capture
#Descomprimimos la ruta donde tendremos el archivo .zip de todas las imagenes del conjunTo de datos seleccionado
#el conjunto de datos estará disponible en github https://github.com/felipkmpo/proyectomlops, recuerde validar su ruta
!unzip "/content/drive/MyDrive/datasetcancerskin1.zip"

**Recuerda descargar el dataset y subirlo a tu entorno colab, revisa bien la ruta para que las etiquetas identifiquen bien las clases**

In [ ]:
#Con os.listdir crearemos una lista de las rutas de una carpeta, esta sera el nombre de las clases a predecir, seran 2 clases
etiquetas_labels = os.listdir("/content/datasetcancerskin")
print(etiquetas_labels)

['malignant', 'benign']


In [ ]:
#Etiquetas labels tiene los nombres de las etiquetas, entonces debemos recorrer cada uno de los archivos dentro de esas etiquetas
names_cero = os.listdir("/content/datasetcancerskin/"+ etiquetas_labels[0])
print(f"Tamaño de {etiquetas_labels[0]}: {len(names_cero)}")
names_uno = os.listdir("/content/datasetcancerskin/"+ etiquetas_labels[1])
print(f"Tamaño de {etiquetas_labels[1]}: {len(names_uno)}")


Tamaño de malignant: 1497
Tamaño de benign: 1800


### TRANSFORMACION DATOS PARA CARGUE EN MODELO PREDICTIVOS

In [ ]:
#Construimos arreglos "X" y "Y", X es la lista de imagenes y y las etiquetas
carpeta = "/content/datasetcancerskin/"

X = []
y = []

etiquetas_labels = os.listdir(carpeta)
#Recorremos los elementos de las etiquetas en orden
for etiqueta in etiquetas_labels:
  #Para cada etiqueta  obtenemos la lista de nombres
  ruta_imagenes = os.listdir(carpeta + etiqueta)
  #Para cada ruta de imagenes leemos la imagén con cv2, la pasamos a RGB y la
  #añadimos a las listas iniciales
  for img in ruta_imagenes:
    imagen = cv2.imread(carpeta + etiqueta + "/" + img)
    imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB) #Puede convertirla a gris

    #Les hacemos un resize para asegurarnos que todas las imagenes tengan el mismo tamaño de entrada
    imagen = cv2.resize(imagen, (100, 100))
    X.append(imagen)
    #Si la etiqueta es la de la primer clase añado un 0, de la clase 2 añado un 1 y de la tercera clase un 2
    if etiqueta == etiquetas_labels[0]:
      y.append(0)
    elif  etiqueta == etiquetas_labels[1]:
      y.append(1)
    elif etiqueta == etiquetas_labels[2]:
      y.append(2)

In [ ]:
# Se trabaja como arreglos de numpy
X = np.array(X)
y = np.array(y)

In [ ]:
#Imprimimos los tamaños de los arreglos
print(X.shape)
print(y.shape)

(3297, 100, 100, 3)
(3297,)


### División Dataset para pruebas y entrenamiento

In [ ]:
#el test_size dice que el 30% de los datos sean de testeo, random_state asegura que cada que
#ejecute el código va a coger las mismas porciones de datos y stratify le asegura que mezcle los
#datos y en todos los arreglos aparezcan de las dos clases
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [ ]:
#Tomamos 10% de la data test para validacion
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.1, random_state=42, stratify=y_test)

In [ ]:
#tamaños de los arreglos despues de la division
print(f'arreglo x entrenamiento: ',X_train.shape)
print(f'arreglo x prueba: ',X_test.shape)
print(f'arreglo x validacion: ',X_val.shape)
print(f'arreglo y entrenamiento: ',y_train.shape)
print(f'arreglo y prueba: ',y_test.shape)
print(f'arreglo y validacion: ',y_val.shape)

arreglo x entrenamiento:  (2307, 100, 100, 3)
arreglo x prueba:  (891, 100, 100, 3)
arreglo x validacion:  (99, 100, 100, 3)
arreglo y entrenamiento:  (2307,)
arreglo y prueba:  (891,)
arreglo y validacion:  (99,)


In [ ]:
# Descripcion de datos por cada clase:
print(np.unique(y_train, return_counts=True))
print(np.unique(y_test, return_counts=True))
print(np.unique(y_val, return_counts=True))

(array([0, 1]), array([1047, 1260]))
(array([0, 1]), array([405, 486]))
(array([0, 1]), array([45, 54]))


### Configuración para rastrear experimentos de TensorFlow con MLflow

In [ ]:
mlflow.tensorflow.autolog()  # Activa el autolog de MLflow para registrar métricas, parámetros y el modelo automáticamente

### NORMALIZAMOS DATOS

In [ ]:
# Reformatear las imágenes para incluir el canal de color
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 3)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 3)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 3)

# Normalizar los datos
X_train = X_train / 255.0
X_test = X_test / 255.0
X_val = X_val / 255.0

# Verificar las formas de los datos
print("Shape of x_train:", X_train.shape)
print("Shape of x_test:", X_test.shape)
print("Shape of x_val:", X_val.shape)

Shape of x_train: (2307, 100, 100, 3)
Shape of x_test: (891, 100, 100, 3)
Shape of x_val: (99, 100, 100, 3)


### Definición y compilación Modelo CNN

**EXPERIMENTO** 1

In [ ]:
#codigo para utilizar wandb
#dentro de config definiremos datos basicos del proyecto icnluyendo numero de epocas para el entrenamietno del modelo
config={
"architecture": "CNN",
"dataset": "imagenes cancer de piel",
"epochs": 10,
}

wandb.init(
      # en el .init estableceremos nombre del proyecto, nombre del experimiento lo que nos servira para luego identificar los experimentos
      project="Proyecto Cancer de Piel",
      name=f"experiment_1",
      config=config
)

#el proceso de registro de dataset solo lo realizaremos una unica vez
# creamos nuevo artefacto para registro dataset en wandb
dataset_artifact = wandb.Artifact("my_dataset", type="dataset")
#guardamos data a arrays
np.save("X_train.npy", X_train)
np.save("y_train.npy", y_train)
np.save("X_test.npy", X_test)
np.save("y_test.npy", y_test)
#agregamos arrays recien creados a artefacto tipo dataset
dataset_artifact.add_file("X_train.npy")
dataset_artifact.add_file("y_train.npy")
dataset_artifact.add_file("X_test.npy")
dataset_artifact.add_file("y_test.npy")
#registramos artefacto en wandb
wandb.log_artifact(dataset_artifact)

#definicion de modelo CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Para clasificación binaria
])

# Compilar el modelo con función de pérdida y optimizador
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**### IMPORTANTE, recuerda ejecutar un experimento a la vez y posteriormente el entrenamiento**

**EXPERIMENTO 2**

In [ ]:
#codigo para utilizar wandb
#dentro de config definiremos datos basicos del proyecto icnluyendo numero de epocas para el entrenamietno del modelo
config={
"architecture": "CNN",
"dataset": "imagenes cancer de piel",
"epochs": 15,
}

wandb.init(
      # en el .init estableceremos nombre del proyecto, nombre del experimiento lo que nos servira para luego identificar los experimentos
      project="Proyecto Cancer de Piel",
      name=f"experiment_02",
      config=config
)

#definicion de modelo CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Para clasificación binaria
])

# Compilar el modelo con función de pérdida y optimizador
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])



epoch/accuracy,▁▆▆▇▇▇█▇▇█
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▃▂▂▂▁▁▁▁▁
epoch/val_accuracy,▆▁▇█▇█▇▇▇█
epoch/val_loss,▇█▃▂▂▁▂▂▃▁
epoch/accuracy,0.81409
epoch/epoch,9
epoch/learning_rate,0.001
epoch/loss,0.3791
epoch/val_accuracy,0.81602


**EXPERIMENTO 3**

In [ ]:
#codigo para utilizar wandb
#dentro de config definiremos datos basicos del proyecto icnluyendo numero de epocas para el entrenamietno del modelo
config={
"architecture": "CNN",
"dataset": "imagenes cancer de piel",
"epochs": 20,
}

wandb.init(
      # en el .init estableceremos nombre del proyecto, nombre del experimiento lo que nos servira para luego identificar los experimentos
      project="Proyecto Cancer de Piel",
      name=f"experiment_03",
      config=config
)

#definicion de modelo CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Para clasificación binaria
])

# Compilar el modelo con función de pérdida y optimizador
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


epoch/accuracy,▁▄▄▅▅▆▆▆▆▆▇▇███
epoch/epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▃▃▂▂▂▂▂▂▂▂▁▁▁▁
epoch/val_accuracy,▁▂▆▆▅▆▇▆█▆███▇▅
epoch/val_loss,█▇▄▃▆▃▃▂▂▄▁▁▂▃█
epoch/accuracy,0.85149
epoch/epoch,14
epoch/learning_rate,0.001
epoch/loss,0.30381
epoch/val_accuracy,0.7987


### ENTRENAMIENTO DEL MODELO

In [ ]:
#entrenamiento del modelo y  registro de la información de la ejecución con MLflow, herramienta de seguimiento de experimentos
if mlflow.active_run():
    mlflow.end_run()
with mlflow.start_run() as run:
    history = model.fit(X_train, y_train,
                        epochs=config["epochs"],
                        validation_split=0.2,
                        batch_size=32,
                        callbacks=[WandbMetricsLogger()]
    )

#PROCESO PARA REGISTRAR DATA SET EN MLFLOW
# Crea una carpeta temporal para el dataset
temp_dir = tempfile.mkdtemp()

# Guarda los datos en la carpeta temporal para este caso utilizaremos numpy
np.save(os.path.join(temp_dir, "X_train.npy"), X_train)
np.save(os.path.join(temp_dir, "y_train.npy"), y_train)
np.save(os.path.join(temp_dir, "X_test.npy"), X_test)
np.save(os.path.join(temp_dir, "y_test.npy"), y_test)

# Registra la carpeta temporal como un artefacto
mlflow.log_artifacts(temp_dir, artifact_path="dataset")
#--------------

# Evaluar precisión en conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test, y_test)

#validacion registro metricas plataformas wandb y mlflow
mlflow.log_metric("accuracy", test_accuracy)
wandb.log({"accuracy": test_accuracy})

# Guardar el modelo en un archivo .h5 con nombre unico
model.save(f"my_model_{wandb.run.id}.h5")

# Registrar el modelo como un artefacto en wandb
artifact = wandb.Artifact(f"my_model_{wandb.run.id}", type="model")
artifact.add_file(f"my_model_{wandb.run.id}.h5")
wandb.log_artifact(artifact)

print(f'\nTest accuracy: {test_accuracy:.4f}')



Epoch 1/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6318 - loss: 1.3809

58/58 ━━━━━━━━━━━━━━━━━━━━ 8s 53ms/step - accuracy: 0.6330 - loss: 1.3706 - val_accuracy: 0.8009 - val_loss: 0.3988
Epoch 2/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.7594 - loss: 0.4548 - val_accuracy: 0.7944 - val_loss: 0.4003
Epoch 3/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7720 - loss: 0.4386

58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7721 - loss: 0.4384 - val_accuracy: 0.8139 - val_loss: 0.3921
Epoch 4/10
57/58 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8002 - loss: 0.3944

58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8006 - loss: 0.3935 - val_accuracy: 0.8355 - val_loss: 0.3594
Epoch 5/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8295 - loss: 0.3505 - val_accuracy: 0.8355 - val_loss: 0.3720
Epoch 6/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8341 - loss: 0.3420 - val_accuracy: 0.8333 - val_loss: 0.3601
Epoch 7/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8636 - loss: 0.3158 - val_accuracy: 0.8268 - val_loss: 0.3602
Epoch 8/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8535 - loss: 0.3038 - val_accuracy: 0.8290 - val_loss: 0.3616
Epoch 9/10
55/58 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8713 - loss: 0.2739

58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8711 - loss: 0.2741 - val_accuracy: 0.8312 - val_loss: 0.3364
Epoch 10/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8668 - loss: 0.2834 - val_accuracy: 0.8333 - val_loss: 0.3441
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 407ms/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.8190 - loss: 0.3695



Test accuracy: 0.8204


### Implementación NGROK para exponer un servidor local para MLFLOW

In [ ]:
#Este comando usa ngrok, una herramienta que permite exponer localmente un servidor a través de un túnel seguro
!ngrok config add-authtoken 2oE5CepTnPeNj0Y1c5fJe6hggaZ_4GHmqaRcCaLLBUbFpUfLd

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import mlflow
from pyngrok import ngrok


# Iniciar el túnel de ngrok para exponer el puerto 5000
ngrok_tunnel = ngrok.connect(5000)


mlflow_url = ngrok_tunnel.public_url
print(f"MLflow Tracking UI: {mlflow_url}")


# Iniciar el servidor de MLflow en el entorno de Colab
get_ipython().system_raw("mlflow ui --host 0.0.0.0 --port 5000 &")

MLflow Tracking UI: https://69a5-35-231-96-38.ngrok-free.app
